In [1]:
import cv2, time, numpy as np, os, tempfile
import numpy as np
from collections import deque
import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.models as models
from typing import List, Tuple
from ultralytics import YOLO

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
# ---------- YOLO detector wrapper ----------
class YoloFaceDetector:
    def __init__(self, weights="yolov12n-face.pt", imgsz=640, conf=0.35, iou=0.5):
        self.model = YOLO(weights)
        self.imgsz = imgsz
        self.conf = conf
        self.iou = iou

    def __call__(self, frame_bgr):
        results = self.model.predict(
            source=frame_bgr, imgsz=self.imgsz,
            conf=self.conf, iou=self.iou, verbose=False, device="cpu"
        )[0]
        dets = []
        if results.boxes is not None and len(results.boxes) > 0:
            xyxy = results.boxes.xyxy.cpu().numpy()
            conf = results.boxes.conf.cpu().numpy()
            for k in range(len(xyxy)):
                x1, y1, x2, y2 = xyxy[k].tolist()
                dets.append({"bbox": [x1, y1, x2, y2], "conf": float(conf[k])})
        return dets



In [26]:
source = "data/dangoon.mp4"

In [58]:
def main():
    model = YOLO("yolov12n-face.pt")   # your face weights
    # model = YOLO("yolo11n-pose.pt")   # your face weights
    win = "YOLO + BoT-SORT (Ultralytics, CPU)"
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)

    try:
        for res in model.track(
            source=source,
            # source=0,
            tracker='botsort_custom.yaml',   # ← built-in config shipped with your installed version
            stream=True,
            imgsz=640,
            conf=0.35,
            iou=0.5,
            device='cpu',
            verbose=False,
            persist=True
        ):
            frame = res.orig_img
            boxes = res.boxes
            if boxes is not None and len(boxes) > 0:
                xyxy = boxes.xyxy.cpu().numpy()
                confs = boxes.conf.cpu().numpy()
                ids = boxes.id.cpu().numpy() if boxes.id is not None else np.full((len(confs),), -1)
                for k, (x1,y1,x2,y2) in enumerate(xyxy.astype(int)):
                    tid  = int(ids[k]) if ids[k] != -1 else k
                    conf = float(confs[k])
                    cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
                    y_text = max(0, y1 - 6)
                    cv2.putText(frame, f"ID {tid}", (x1, y_text),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                    # txt = f"Conf {conf:.2f}"
                    # (tw,_),_ = cv2.getTextSize(txt, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    # cv2.putText(frame, txt, (x2 - tw, y_text),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
            cv2.imshow(win, frame)
            k = cv2.waitKey(1) & 0xFF
            if k in (27, ord('q')) or cv2.getWindowProperty(win, cv2.WND_PROP_VISIBLE) < 1:
                break
    finally:
        cv2.destroyWindow(win)
        for _ in range(3): cv2.waitKey(1)
        time.sleep(0.05)


In [60]:
if __name__ == "__main__":
    main()